In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp submission
# all_local

In [ ]:
#export
import enum
import glob
import os
from sqlalchemy import Table, Column, Integer, Enum, JSON, Text
from steroidsornot.database import Base, Session

In [ ]:
#export
class Label(enum.Enum):
    natural = 1
    steroids = 2
    uncertain = 3
    irrelevant = 4
    deleted = 5
    autoremoved = 6

In [ ]:
#export
class Submission(Base):
    __tablename__ = "submission"

    id = Column(Integer, primary_key=True)
    label = Column(Enum(Label))
    data = Column(JSON)
    image_path = Column(Text)

    def __repr__(self):
        return f"Submission(id={self.id!r}, label={self.label!r}, data={self.data!r})"

In [ ]:
def synchronize_image_paths(session):
    posts = session.query(Submission).all()
    for index, post in enumerate(posts):
        path = post.image_path
        if path:
            filename = os.path.basename(path)
            new_path = glob.glob(f'/home/cedar/source/steroids-or-not/data/images/**/{filename}')[0]
            post.image_path = new_path
            session.add(post)
            
        # else don't worry about it, if there's no stored path there's nothing to update
        
    session.commit()